In [1]:
import sys
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
import torch as t
t.autograd.set_grad_enabled(False)
from tqdm import tqdm
from glob import glob
import pandas as pd
import numpy as np
import re
from transformers import RobertaTokenizerFast, DebertaModel
import dill as pickle

In [2]:
class TvmLongformer(t.nn.Module):
    def __init__(self):
        super().__init__()
        self.feats = DebertaModel.from_pretrained(
            'microsoft/deberta-large')
        self.feats.pooler = None
        self.class_projector = t.nn.Sequential(
            t.nn.LayerNorm(1024),
            t.nn.Linear(1024, 15)
        )
    def forward(self, tokens, mask):
        return self.class_projector(self.feats(tokens, mask, return_dict=False)[0])
    
model = TvmLongformer()

In [3]:
class Dataset(t.utils.data.Dataset):
    def __init__(self, files):
        tokenizer = RobertaTokenizerFast.from_pretrained('roberta-large')
        tokenizer.model_max_length = 4096
        self.tokenizer = tokenizer
        files = set(files)
        self.texts = {}
        for file_id in files:
            with open(f'train/{file_id}.txt') as f:
                self.texts[file_id] = f.read().strip()
        self.keys = sorted(self.texts.keys())
        self.space_regex = re.compile('[\s\n]')
    def __len__(self):
        return len(self.keys)
    def __getitem__(self, ix):
        tokens_array = np.zeros(4096, 'i8')
        mask_array = np.zeros(4096, 'f4')
        offsets_array = np.zeros((4096, 2), 'i4')
        
        text = self.texts[self.keys[ix]]
        key = self.keys[ix]
        tokenizer_outs = self.tokenizer(text, return_offsets_mapping=True)
        tokens = np.array(tokenizer_outs['input_ids'], 'i8')
        tokens[0] = 1
        mask = np.array(tokenizer_outs['attention_mask'], 'f4')
        offsets = np.vstack(tokenizer_outs['offset_mapping']).astype('i4')
        
        tokens_array[:len(tokens)] = tokens
        mask_array[:len(tokens)] = mask
        offsets_array[:len(tokens)] = offsets
        
        index_map = []
        current_word = 0
        blank = False
        for char_ix in range(text.index(text.strip()[0]), len(text)):
            if self.space_regex.match(text[char_ix]) is not None:
                blank = True
            elif blank:
                current_word += 1
                blank = False
            index_map.append(current_word)
            
        return tokens_array, mask_array, offsets_array, index_map, key, len(tokens)
    
first_batch = True
def collate_fn(ins):
    global first_batch
    if first_batch:
        max_len = 2048
        first_batch = False
    else:
        max_len = (max(x[-1] for x in ins) + 7) // 8 * 8
    return tuple(t.from_numpy(np.concatenate([ins[z][x][None, :max_len]
                                              for z in range(len(ins))]))
                 for x in range(len(ins[0]) - 3)) \
                 + ([x[-3] for x in ins], [x[-2] for x in ins], np.array([x[-1] for x in ins]),)    


label_names = ['None', 'Lead', 'Position', 'Evidence', 'Claim',
               'Concluding Statement', 'Counterclaim', 'Rebuttal']


In [4]:
checkpoints = glob('checkpoints_large_v1/extra_distclean_mse*')

In [ ]:
#sort by fold
#sorted_checkpoints = sorted(checkpoints, key=lambda x: int(x.split('/')[-1].split('_')[3][-1]))

In [5]:
len(checkpoints)

5

In [6]:
model.eval().cuda();

In [7]:
with open('id_to_ix_map.pickle', 'rb') as f:
    id_to_ix_map = {x.split('/')[-1].split('.')[0]: y for x, y in pickle.load(f).items()}
with open('data_splits.pickle', 'rb') as f:
    data_splits = pickle.load(f)

In [8]:
all_outs = np.zeros((len(glob('train/*.txt')), 2048, 15), 'f4')
all_bounds = np.zeros((len(glob('train/*.txt')), 2048, 2), 'i4')
all_token_nums = np.zeros((len(glob('train/*.txt')),), 'i4')
all_word_indices = []
all_sample_ids = []
ix = 0
for fold_ix, checkpoint in enumerate(sorted_checkpoints):
    val_files = data_splits[0][250]['normed'][fold_ix]
    dataset = t.utils.data.DataLoader(Dataset(val_files), collate_fn=collate_fn,
                                  batch_size=1, num_workers=2)
    model.load_state_dict(t.load(checkpoint), strict=False);
    for batch in tqdm(dataset):
        tokens, mask, bounds, word_indices, sample_ids, num_tokens = batch
        num_tokens = num_tokens[0]
        batch_size, batch_len = tokens.shape[:2]
        outs = t.log_softmax(model(tokens.cuda(), mask.cuda()), -1)
        all_outs[ix: ix + batch_size, :num_tokens - 2] = outs[:, 1:num_tokens - 1].cpu().numpy()
        all_bounds[ix: ix + batch_size, :num_tokens - 2] = bounds[:, 1:num_tokens - 1]
        all_token_nums[ix: ix + batch_size] = num_tokens - 2
        all_word_indices.extend(word_indices)
        all_sample_ids.extend(sample_ids)
        ix += batch_size

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 3098/3098 [02:16<00:00, 22.68it/s]


In [ ]:
model_name = 

In [ ]:
if not os.path.exists(f'oof_ps/{model_name}'):
    os.makedirs(f'oof_ps/{model_name}')

In [9]:
for (array, 
    array_name) in zip((all_outs, all_bounds, all_token_nums, all_word_indices, all_sample_ids),
                        'all_outs, all_bounds, all_token_nums, all_word_indices, all_sample_ids'.split(', ')):
    np.save(f'oof_ps/{model_name}/{array_name}.npy', array)

/home/qwe/miniconda3/envs/pytorch2/lib/python3.7/site-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)
